## Exercise:
1. Find the week (and its associated year) with the highest total weekly consumption.

2. Find the day of the week that averages the highest consumption

3. Find the time of day that averages the lowest consumption.
    - Has this changed over the years?
    

4. Is average consumption rising, falling, or staying the same over the years?
5. What is the %age difference in consumption on average between April and June?

In [3]:
import pandas as pd

In [6]:
energy = pd.read_csv('../data/energy/PJM_Load_hourly.csv', 
                     parse_dates=True, index_col=0)

In [7]:
indexes = [energy.index.year,
           energy.index.month,
           energy.index.week,
           energy.index.weekday,
           energy.index.day]

aggregated = energy.groupby(indexes).sum()

In [8]:
index_level_names = ['year', 'month', 'week', 'weekday', 'day']
for i, index_level in enumerate(index_level_names):
            aggregated.index.set_names(index_level, level=i, inplace=True)

1. Find the week (and its associated year) with the highest total weekly consumption.

In [9]:
aggregated_col = aggregated.reset_index()
week_max = aggregated_col.groupby('week').max()
week_max.head(1)

,year,month,weekday,day,PJM_Load_MW
week,,,,,
1,2002,12,6,31,867950.0


2. Find the day of the week that averages the highest consumption

NOTE: Weekdays in Pandas are listed from 0-6, with Monday as 0 and Sunday as 6. Thus, 2 is Wednesday.

In [17]:
aggregated.groupby('weekday').mean().sort_values("PJM_Load_MW", ascending = False).iloc[0].name

2

3. Find the time of day that averages the lowest consumption.
    - Has this changed over the years?

In [23]:
# for all years
time_aggregated = energy.groupby(energy.index.hour).mean()
time_aggregated.idxmin()

PJM_Load_MW    4
dtype: int64

In [34]:
# looking at the change over years
time_aggregated = energy.groupby([energy.index.year, energy.index.hour]).mean()
index_level_names = ['year', 'hour']
for i, index_level in enumerate(index_level_names):
            time_aggregated.index.set_names(index_level, level=i, inplace=True)

PJM_Load_MW    (1998, 4)
dtype: object

It is changing over time- in the last year, midnight is the lowest use hour.

In [44]:
time_aggregated.groupby(level=0).idxmin().values

array([[(1998, 4)],
       [(1999, 4)],
       [(2000, 4)],
       [(2001, 4)],
       [(2002, 0)]], dtype=object)

4. Is average consumption rising, falling, or staying the same over the years?

In [39]:
yearly_avg_consumption = energy.groupby(energy.index.year).mean()
yearly_avg_consumption

,PJM_Load_MW
Datetime,
1998,28761.826436
1999,29642.061772
2000,30114.333182
2001,30298.447933
2002,31569.000000


Average consumption is increasing over the years.

5. What is the %age difference in consumption on average between April and June?

In [46]:
avg_monthly_consumption = energy.groupby(energy.index.month).mean()
april_consump = avg_monthly_consumption.loc[4]
june_consump = avg_monthly_consumption.loc[6]
percent_diff = ((june_consump - april_consump) / april_consump ) * 100
print('April consumption average is:', int(april_consump))
print('June consumption average is:', int(june_consump))
print('Percentage Difference in consumption average is: {}%'.format(int(percent_diff)))

April consumption average is: 26104
June consumption average is: 31703
Percentage Difference in consumption average is: 21%
